# **DATA WRANGLING SITUACIÓN ESTACIONES**
This file attemps to gather, collect, and transform the station situations raw dataset from the source attached below in order to analyse the data avilable and proceed with it. The following processes will be dealt with:

1. Reading the .json files and transforming variables
2. Data exploration
3. Reshaping data
4. Filtering data

Source: https://datos.madrid.es/sites/v/index.jsp?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&buscar=true&Texto=BiciMAD&Sector=&Formato=&Periodicidad=&orderByCombo=CONTENT_INSTANCE_NAME_DECODE

#### **LIBRARIES**

In [89]:
import pandas as pd 
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

#### **1. READ DATA and VARIABLE TRANSFORMATION**
**Dataset**: 20190x.json (X stands for the month number)    

**Description**: Dataset of the situation of the different stations of BiciMAD by date and time

**Dataframe size**: Each month 145,286 timestamps are colected with information on 14 variables. Observations froom july to december total 883,777 timestamps

**Variables**:
- _id_: Código de la Estación Base
- _latitude_: Latitud de la estación en formato WGS84
- _longitude_: Longitud de la Estación en formato WGS84
- _name_: Nombre de la Estación
- _light_: Grado de Ocupación (0=baja, 1=media, 2=alta, 3=inactiva)
- _number_: Denominación lógica de la Estación Base
- _actívate_: Estación activa (0=No activa, 1=activa)
- _no_available_: Disponibilidad de la Estación (0=disponible, 1=no disponible)
- _total_bases_: Número de bases de la estación
- _dock_bikes_: Número de bicicletas ancladas
- _free_bases_: Número de bases libres
- _reservations_count_: Número de reservas activas

In [2]:
itinerarios_list = ["201907", "201908", "201909", "201910", "201911", "201912"]

situaciones = pd.DataFrame()

for i in itinerarios_list:
    with open('../Data/Situacion Estaciones/'+i+'.json','r') as f:
        data = json.loads(f.read())

    df_i = pd.json_normalize(
        data, 
        meta=['_id'],
        record_path =['stations']
    )
    df_i["_id"] = pd.to_datetime(df_i["_id"])
    situaciones = pd.concat( [situaciones, df_i.drop(columns="id")], ignore_index=True, axis=0)

situaciones.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,_id
0,1,Puerta del Sol A,0,2,24,15,1a,-3.7024255,0,Puerta del Sol nº 1,40.4168961,8,2019-07-01 00:29:26.018083
1,1,Puerta del Sol B,0,2,24,14,1b,-3.7024207,0,Puerta del Sol nº 1,40.4170009,9,2019-07-01 00:29:26.018083
2,1,Miguel Moya,0,2,24,12,2,-3.7058415,0,Calle Miguel Moya nº 1,40.4205886,10,2019-07-01 00:29:26.018083
3,1,Plaza Conde Suchil,0,1,18,4,3,-3.7069171,0,Plaza del Conde Suchil nº 2-4,40.4302937,11,2019-07-01 00:29:26.018083
4,1,Malasaña,0,0,24,17,4,-3.7025875,0,Calle Manuela Malasaña nº 5,40.4285524,3,2019-07-01 00:29:26.018083


In [3]:
situaciones.shape

(883777, 13)

**Variables type check**: correct

In [4]:
situaciones.dtypes

activate                       int64
name                          object
reservations_count             int64
light                          int64
total_bases                    int64
free_bases                     int64
number                        object
longitude                     object
no_available                   int64
address                       object
latitude                      object
dock_bikes                     int64
_id                   datetime64[ns]
dtype: object

**NaN check**: don't exist

In [5]:
situaciones.isna().sum()

activate              0
name                  0
reservations_count    0
light                 0
total_bases           0
free_bases            0
number                0
longitude             0
no_available          0
address               0
latitude              0
dock_bikes            0
_id                   0
dtype: int64

In [3]:
x = situaciones[situaciones["number"]=="10"]

In [4]:
x.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,_id
10,1,Marqués de la Ensenada,0,0,24,22,10,-3.6918807,0,Calle Marqués de la Ensenada nº 16,40.4250863,2,2019-07-01 00:29:26.018083
194,1,Marqués de la Ensenada,0,0,24,21,10,-3.6918807,0,Calle Marqués de la Ensenada nº 16,40.4250863,3,2019-07-01 03:29:30.886274
378,1,Marqués de la Ensenada,0,0,24,23,10,-3.6918807,0,Calle Marqués de la Ensenada nº 16,40.4250863,1,2019-07-01 04:29:32.858592
562,1,Marqués de la Ensenada,0,0,24,23,10,-3.6918807,0,Calle Marqués de la Ensenada nº 16,40.4250863,1,2019-07-01 05:29:34.913030
746,1,Marqués de la Ensenada,0,0,24,23,10,-3.6918807,0,Calle Marqués de la Ensenada nº 16,40.4250863,1,2019-07-01 06:29:35.163470


In [20]:
x["light"].value_counts()

0    2619
2     942
1     775
Name: light, dtype: int64

In [88]:
def SetColorLight(c):
    if(c < 1):
        return "green"
    elif(c >= 1 | c <2 ):
        return "yellow"
    elif(c >= 2 | c <3 ):
        return "orange"
    elif(c >= 3):
        return "red"

In [87]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agrego las trazas necesarias
fig.add_trace(
    go.Bar(
        x = x["_id"],
        y =x.groupby(x["_id"].dt.hour)["light"].mean(),
        marker =dict(color=list(map(SetColorLight, x["light"]))),
        opacity = 0.2,
        name = "Light (Grado de ocupación)"
    ),
        secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x.groupby(x["_id"].dt.hour)["reservations_count"].mean(),
        mode = "lines",
        line = dict(color = "steelblue"),
        name = "Reservations (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x.groupby(x["_id"].dt.hour)["free_bases"].mean(),
        mode = "lines",
        line = dict(color = "green", dash = 'dash'),
        name = "Free bases (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x.groupby(x["_id"].dt.hour)["total_bases"].mean(),
        mode = "lines",
        line = dict(color = "purple", dash = 'dash'),
        name = "Total bases (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x.groupby(x["_id"].dt.hour)["dock_bikes"].mean(),
        mode = "lines",
        marker_color = x.groupby(x["_id"].dt.hour)["light"].mean(),
        name = "Dock bikes (Nº de bicletas ancladas)"
    )
)

# Actualizo el diseño
fig.update_layout(title = "Estado de la base 10 por horas", xaxis_title = "Hour of the day" , yaxis_title = "Number of <b>Bases</b>")
fig.update_yaxes(
        title_text="<b>Light</b> (level of occupation)", 
        secondary_y=True)

# Muestro la figura
fig.show()

In [90]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agrego las trazas necesarias
fig.add_trace(
    go.Bar(
        x = situaciones["_id"],
        y =situaciones.groupby(situaciones["_id"].dt.hour)["light"].mean(),
        marker =dict(color=list(map(SetColorLight, situaciones["light"]))),
        opacity = 0.2,
        name = "Light (Grado de ocupación)"
    ),
        secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x = situaciones["_id"],
        y =situaciones.groupby(situaciones["_id"].dt.hour)["reservations_count"].mean(),
        mode = "lines",
        line = dict(color = "steelblue"),
        name = "Reservations (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = situaciones["_id"],
        y =situaciones.groupby(x["_id"].dt.hour)["free_bases"].mean(),
        mode = "lines",
        line = dict(color = "green", dash = 'dash'),
        name = "Free bases (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = situaciones["_id"],
        y =situaciones.groupby(situaciones["_id"].dt.hour)["total_bases"].mean(),
        mode = "lines",
        line = dict(color = "purple", dash = 'dash'),
        name = "Total bases (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = situaciones["_id"],
        y =situaciones.groupby(situaciones["_id"].dt.hour)["dock_bikes"].mean(),
        mode = "lines",
        marker_color = situaciones.groupby(situaciones["_id"].dt.hour)["light"].mean(),
        name = "Dock bikes (Nº de bicletas ancladas)"
    )
)

# Actualizo el diseño
fig.update_layout(title = "Estado de la base 10 por horas", xaxis_title = "Hour of the day" , yaxis_title = "Number of <b>Bases</b>")
fig.update_yaxes(
        title_text="<b>Light</b> (level of occupation)", 
        secondary_y=True)

# Muestro la figura
fig.show()

In [ ]:
# Creo el objeto vacío
fig = go.Figure()

# Agrego las trazas necesarias
fig.add_trace(
    go.Bar(
        x = x["_id"],
        y =x["light"],
        
        marker_color = "red",
        name = "Light (Grado de ocupación)"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x["reservations_count"],
        mode = "lines",
        line = dict(color = "steelblue", dash = 'dash'),
        name = "Reservations (Nº de reservas activas)"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x["free_bases"],
        mode = "lines",
        line = dict(color = "green", dash = 'dash'),
        name = "Free bases"
    )
)

fig.add_trace(
    go.Scatter(
        x = x["_id"],
        y =x["dock_bikes"],
        mode = "lines",
        marker_color = x.groupby(x["_id"].dt.hour)["light"].mean(),
        name = "Dock bikes (Nº de bicletas ancladas)"
    )
)

# Actualizo el diseño
fig.update_layout(title = "Estado de las bases por horas", xaxis_title = "Año y mes", yaxis_title = "Pasajeros")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Muestro la figura
fig.show()

In [11]:
situaciones.groupby("number", group_keys=True).apply(lambda x: x).head()

activate                    name  reservations_count  light  \
number                                                                    
10     10          1  Marqués de la Ensenada                   0      0   
       194         1  Marqués de la Ensenada                   0      0   
       378         1  Marqués de la Ensenada                   0      0   
       562         1  Marqués de la Ensenada                   0      0   
       746         1  Marqués de la Ensenada                   0      0   

            total_bases  free_bases number   longitude  no_available  \
number                                                                 
10     10            24          22     10  -3.6918807             0   
       194           24          21     10  -3.6918807             0   
       378           24          23     10  -3.6918807             0   
       562           24          23     10  -3.6918807             0   
       746           24          23     10  -3.6918807             0   

                                       address    latitude  dock_bikes  \
number                                                                   
10     10   Calle Marqués de la Ensenada nº 16  40.4250863           2   
       194  Calle Marqués de la Ensenada nº 16  40.4250863           3   
       378  Calle Marqués de la Ensenada nº 16  40.4250863           1   
       562  Calle Marqués de la Ensenada nº 16  40.4250863           1   
       746  Calle Marqués de la Ensenada nº 16  40.4250863           1   

                                  _id  
number                                 
10     10  2019-07-01 00:29:26.018083  
       194 2019-07-01 03:29:30.886274  
       378 2019-07-01 04:29:32.858592  
       562 2019-07-01 05:29:34.913030  
       746 2019-07-01 06:29:35.163470

**############################################TO TRY SINGLE MONTH###########################**

In [8]:
# load data using Python JSON module
with open('../Data/Situacion Estaciones/201909.json','r') as f:
    data = json.loads(f.read())

sit_201909 = pd.json_normalize(
    data, 
    meta=['_id'],
    record_path =['stations']
)
sit_201909["_id"] = pd.to_datetime(sit_201909["_id"])
sit_201909 = sit_201909.drop(columns="id")
sit_201909.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,_id
0,1,Puerta del Sol A,0,0,24,17,1a,-3.7024255,0,Puerta del Sol nº 1,40.4168961,3,2019-09-01 00:10:32.593867
1,1,Puerta del Sol B,0,2,24,13,1b,-3.7024207,0,Puerta del Sol nº 1,40.4170009,10,2019-09-01 00:10:32.593867
2,1,Miguel Moya,0,2,24,8,2,-3.7058415,0,Calle Miguel Moya nº 1,40.4205886,16,2019-09-01 00:10:32.593867
3,1,Plaza Conde Suchil,0,0,18,12,3,-3.7069171,0,Plaza del Conde Suchil nº 2-4,40.4302937,3,2019-09-01 00:10:32.593867
4,1,Malasaña,0,2,24,8,4,-3.7025875,0,Calle Manuela Malasaña nº 5,40.4285524,12,2019-09-01 00:10:32.593867
